# Higgs To 4 Leptons Analysis with Systematic Uncertainties Using uproot, awkward and pyhf

## Imports and Variable Initialization

This is where we state the location of our .root files. To reduce the memory usage, we select which columns of the .root files will be used when building our dataframes.

In [ ]:
# original: https://root.cern.ch/doc/v622/df106__HiggsToFourLeptons_8py.html
from collections import Counter
from itertools import compress

import os
import math
import json
import glob

from pprint import pprint as pp

from datetime import datetime
t1 = datetime.now()
print(t1)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import uproot4 as uproot
from uproot_methods import TLorentzVector
import awkward1 as ak

path = "root://eospublic.cern.ch//eos/opendata/atlas/OutreachDatasets/2020-01-22"
path = "/home/zbhatti/codebase/intro-to-hep/OutreachDatasets/2020-01-22"
files = json.load(open(os.path.join(os.environ["ROOTSYS"], "tutorials/dataframe", "df106_HiggsToFourLeptons.json")))

pp(files)
use_cached = True
lumi = 10064.0

read_columns = ['trigE', 'trigM', 'lep_eta', 'lep_pt', 'lep_ptcone30', 'lep_etcone20', 
                'lep_type', 'lep_charge', 'lep_phi', 'lep_E', 'lep_trackd0pvunbiased', 
                'lep_tracksigd0pvunbiased', 'lep_z0', 'mcWeight', 'scaleFactor_ELE', 
                'scaleFactor_MUON', 'scaleFactor_LepTRIGGER', 'scaleFactor_PILEUP',
               ]

dataframes_npy_filename = 'higgs_4l_dataframes-{}-.npy'
cut_counts_npy_filename = 'higgs_4l_cut_counts-{}-.npy'

processes = files.keys()
dataframes = {}
dataframes_high = {}
dataframes_low = {}
xsecs = {}
sumws = {}
samples = []
cut_counts = {}


## Load .root Files and Apply Cuts

Here we apply cuts, reducing the total number of events we eventually bin into histograms. Using existing or defining new columns helps us make the right choice of filters to apply.

In [ ]:
def lorentz_vector(pt, eta, phi, e):
    return TLorentzVector.from_ptetaphie(pt, eta, phi, e)

# https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-448 for [*map()] behavior
def lorentz_vectors(pt_arr, eta_arr, phi_arr, e_arr):
    return [*map(TLorentzVector.from_ptetaphie, pt_arr, eta_arr, phi_arr, e_arr)]

# cuts based on lepton flavor
def good_electrons_and_muons(pdg_id_arr, lv_arr, trackd0pv_arr, tracksigd0pv_arr, z0_arr):
    for i, pdg_id in enumerate(pdg_id_arr):
        if pdg_id == 11:
            if (lv_arr[i].pt < 7000 or abs(lv_arr[i].eta) > 2.47 or abs(trackd0pv_arr[i] / tracksigd0pv_arr[i]) > 5 or abs(z0_arr[i] * math.sin(lv_arr[i].theta)) > 0.5):
                return False
        else:
            if (abs(trackd0pv_arr[i] / tracksigd0pv_arr[i]) > 5 or abs(z0_arr[i] * math.sin(lv_arr[i].theta)) > 0.5):
                return False
    return True

# works on ALL leptons in all events, or per event
def good_lep(lep_eta, lep_pt, lep_ptcone30, lep_etcone20):
    a = np.logical_and(abs(lep_eta) < 2.5, lep_pt > 5000)
    b = np.logical_and(lep_ptcone30 / lep_pt < 0.3, lep_etcone20 / lep_pt < 0.3)
    return np.logical_and(a, b)
    
def invariant_mass(lep_lv_arr):
    return math.sqrt((lep_lv_arr[0] + lep_lv_arr[1] + lep_lv_arr[2] + lep_lv_arr[3]).mass2) / 1000.0;

def build_dataframe(tree, xsecs_sample, sumws_sample, data_or_mc, threshold):
    dataframes = None
    cut_counts = np.array([0, 0, 0, 0, 0, 0, 0])
    
    # Dummy experimental uncertainty 
    if 'high' == threshold:
        pt_threshold = 12000
    elif 'low' == threshold:
        pt_threshold = 8000
    else:
        pt_threshold = 10000
    
    for df in tree.iterate(read_columns, library='ak'):
            sample_batch_counts = [df.shape[0]]
            
            # Filter 1
            df = df[df.trigE | df.trigM]
            sample_batch_counts.append(df.shape[0])
            
            # can call on ALL events
            df['good_lep'] = good_lep(df.lep_eta, df.lep_pt, df.lep_ptcone30, df.lep_etcone20)
            
            # or PER event (slower)
            # df['good_lep'] = [*map(good_lep, df.lep_eta, df.lep_pt, df.lep_ptcone30, df.lep_etcone20)]

            # Filter 2
            df = df[np.sum(df.good_lep, axis=1) == 4]
            sample_batch_counts.append(df.shape[0])
            
            # Filter 3
            df = df[np.sum(df.lep_charge[df.good_lep], axis=1) == 0]
            sample_batch_counts.append(df.shape[0])

            df['goodlep_sumtypes'] = np.sum(df.lep_type[df.good_lep], axis=1)
            
            # Filter 4
            df = df[(df.goodlep_sumtypes == 44) | (df.goodlep_sumtypes == 52) | (df.goodlep_sumtypes == 48)]
            sample_batch_counts.append(df.shape[0])
            
            df['goodlep_pt'] = df.lep_pt[df.good_lep]
            df['goodlep_eta'] = df.lep_eta[df.good_lep]
            df['goodlep_phi'] = df.lep_phi[df.good_lep]
            df['goodlep_E'] = df.lep_E[df.good_lep]

            goodlep_lv = [*map(lorentz_vectors, 
                                ak.to_numpy(df.goodlep_pt),
                                ak.to_numpy(df.goodlep_eta),
                                ak.to_numpy(df.goodlep_phi),
                                ak.to_numpy(df.goodlep_E)
                               )]

            # Filter 5           
            good_e_mu_filter = [*map(good_electrons_and_muons,
                                     ak.to_numpy(df.lep_type[df.good_lep]),
                                     goodlep_lv,
                                     ak.to_numpy(df.lep_trackd0pvunbiased[df.good_lep]),
                                     ak.to_numpy(df.lep_tracksigd0pvunbiased[df.good_lep]),
                                     ak.to_numpy(df.lep_z0[df.good_lep])
                                    )]
            df = df[good_e_mu_filter]
            good_e_mu_lv = [*compress(goodlep_lv, good_e_mu_filter)]
            sample_batch_counts.append(df.shape[0])
            
            # Filter 6
            high_pt_filter = [*map(lambda x: x[0] > 25000 and x[1] > 15000 and x[2] > pt_threshold, df.goodlep_pt)]
            df = df[high_pt_filter]
            high_pt_lv = [*compress(good_e_mu_lv, high_pt_filter)]

            sample_batch_counts.append(df.shape[0])
            print(sample_batch_counts)
            
            df['m4l'] = [*map(invariant_mass, high_pt_lv)]
                        
            if "data" in data_or_mc:
                df['weight'] = np.ones(df.shape[0]) # DOUBLE CHECK THESE ARE FLOATS!
            
            elif "mc" in data_or_mc:
                df['weight'] = df.scaleFactor_ELE * df.scaleFactor_MUON * \
                               df.scaleFactor_LepTRIGGER * df.scaleFactor_PILEUP * \
                               df.mcWeight * (xsecs_sample / sumws_sample) * lumi
            
            # free up memory       
            df = df[['m4l', 'weight']]
            cut_counts = np.array(sample_batch_counts) + cut_counts
            
            if dataframes is None: 
                dataframes = df
                
            else:
                dataframes = dataframes.append(df) 
        
    return dataframes, cut_counts

In [ ]:
for p in processes:
    for d in files[p]:
        # Construct the dataframes
        folder = d[0] # Folder name
        sample = d[1] # Sample name
        xsecs[sample] = d[2] # Cross-section
        sumws[sample] = d[3] # Sum of weights
        samples.append(sample)
        filepath = "{}/4lep/{}/{}.4lep.root".format(path, folder, sample)
        print(sample, filepath)
        uproot_file = uproot.open(filepath)
        tree = uproot_file['mini']
        data_or_mc = "data" if "data" in sample.lower() else "mc"
        
        if not use_cached:
            dataframes[sample], cut_counts[sample] = build_dataframe(tree, xsecs[sample], sumws[sample], data_or_mc, None)
            dataframes_high[sample], _ = build_dataframe(tree, xsecs[sample], sumws[sample], data_or_mc, 'high')
            dataframes_low[sample], _ = build_dataframe(tree, xsecs[sample], sumws[sample], data_or_mc, 'low')
            np.save(dataframes_npy_filename.format(sample), ak.to_numpy(dataframes[sample]))
            np.save(dataframes_npy_filename.format(sample+'.high'), ak.to_numpy(dataframes_high[sample]))
            np.save(dataframes_npy_filename.format(sample+'.low'), ak.to_numpy(dataframes_low[sample]))
            np.save(cut_counts_npy_filename.format(sample), cut_counts[sample])
            
            del dataframes[sample]
            del dataframes_high[sample]
            del dataframes_low[sample]


## Load a Previous Run of the Cuts

Fortunately, the work done in the previous steps are a checkpoint. We can pick up our analysis here if we do not need to update the cuts

In [ ]:
pickle_filenames = glob.glob(dataframes_npy_filename.format('*'))
if use_cached:
    for filename in pickle_filenames:
        sample = filename.split('-')[2]
        print('loading sample {}'.format(sample))
        if sample.endswith('.high'):
            dataframes_high[sample[:-5]] = np.load(filename)
        elif sample.endswith('.low'):
            dataframes_low[sample[:-4]] = np.load(filename)
        else:
            dataframes[sample] = np.load(filename)
        cut_counts[sample] = np.load(cut_counts_pickle_filename.format(sample))
        samples.append(sample)

print('Final Shapes for Samples:')
pp(cut_counts)

## Efficiency of the Cuts

Here we see how the cuts reduce background events while not excluding potential signal events

In [ ]:
cut_efficiency = pd.DataFrame()

# Merge Signal and BG counts
cut_efficiency['Signal N'] = cut_counts['mc_345060.ggH125_ZZ4lep'] + cut_counts['mc_344235.VBFH125_ZZ4lep']
cut_efficiency['Background N'] = cut_counts['mc_361106.Zee'] + cut_counts['mc_361107.Zmumu'] + cut_counts['mc_363490.llll']
cut_efficiency['Signal %'] = np.full((cut_counts['mc_361106.Zee'].shape[0], ), np.nan)
cut_efficiency['Background %'] = np.full((cut_counts['mc_361106.Zee'].shape[0], ), np.nan)

# Produce fractional numbers
for region_name in ['Signal', 'Background']:
    region_pct = region_name + ' %'
    region_n = region_name + ' N'
    for filter_i, count in enumerate(cut_efficiency[region_n]):
    
        if filter_i == 0:
            continue
        else:
            prev = cut_efficiency[region_n][filter_i-1]
            curr = cut_efficiency[region_n][filter_i]
            cut_efficiency[region_pct][filter_i] = abs(prev - curr) / prev * 100.

print('events passing each cut and percent difference from prev filter')
pp(cut_efficiency)

## Merge Samples Into Background, Signal and Data Sets

Initially, our labeled data sets are split across multiple .root files

In [ ]:
# merge with bin, low and high parameters taken from first histogram
def merge_series(label):
    x = None
    weights = None
    for i, d in enumerate(files[label]):
        sample = d[1]
        print('merging {}...'.format(sample))
        sample_x = dataframes[sample].m4l.to_numpy()
        sample_weights = dataframes[sample].weight.to_numpy()
        if i == 0: 
            x = sample_x
            weights = sample_weights
        else: 
            x = np.concatenate((x, sample_x), axis=None)
            weights = np.concatenate((weights, sample_weights), axis=None)
    return x, weights

data_x, data_weights = merge_series("data")
higgs_x, higgs_weights = merge_series("higgs")
zz_x, zz_weights = merge_series("zz")
other_x, other_weights = merge_series("other")

dataframes = {} # free up some precious memory

## Build Histograms

Now that all our labeled sets are complete, we can start building histograms and finding the statistical uncertainties for each bin

In [ ]:
n_bins = 24
bin_range = (80, 170)
bin_edges = np.linspace(bin_range[0], bin_range[1], n_bins)
center_offset = (bin_range[1] - bin_range[0]) / n_bins /2. 
bin_centers = np.linspace(bin_range[0]+center_offset, bin_range[1]-center_offset, n_bins)

def extract_stat_uncertainty(x, x_weights, x_hist):
    # get indices of which bins x belong to; np.digitize() returns bin id rather than bin index position
    x_bin_ids = np.digitize(x, bins=x_hist[1]) 
    
    weights_by_bin = [np.array([0.,]),]*len(x_hist[0]) # an empty bin has 0.0 stat uncertainty
    
    # first group weights by their corresponding x bins
    for i, bin_id in enumerate(x_bin_ids):
        if bin_id == 0 or bin_id == n_bins+1: # less than left edge or larger than right edge
            continue
        bin_index = bin_id - 1
        corresponding_weight = x_weights[i]
        if np.sum(weights_by_bin[bin_index]) == 0.:
            weights_by_bin[bin_index] = np.array([corresponding_weight, ])
        
        else:
            weights_by_bin[bin_index] = np.concatenate((weights_by_bin[bin_index], np.array([corresponding_weight, ])), axis=None)
    
    # now calculate statistical uncertainty per bin
    return np.array([np.sqrt(np.sum(bin_weights**2)) for bin_weights in weights_by_bin])

data_hist = np.histogram(data_x, bins=n_bins, range=bin_range, weights=data_weights)
higgs_hist = np.histogram(higgs_x, bins=n_bins, range=bin_range, weights=higgs_weights)
zz_hist = np.histogram(zz_x, bins=n_bins, range=bin_range, weights=zz_weights)
other_hist = np.histogram(other_x, bins=n_bins, range=bin_range, weights=other_weights)

higgs_hist_err = extract_stat_uncertainty(higgs_x, higgs_weights, higgs_hist)
zz_hist_err = extract_stat_uncertainty(zz_x, zz_weights, zz_hist)
other_hist_err = extract_stat_uncertainty(other_x, other_weights, other_hist)

print('higgs, zz, other stat errors:')
pp(higgs_hist_err)
pp(zz_hist_err)
pp(other_hist_err)

print('Histograms:')
print('bins:')
pp(bin_edges)

print('data:')
pp(data_hist[0])

print('higgs:')
pp(higgs_hist[0])

print('zz:')
# Apply MC correction for ZZ due to missing gg->ZZ process
zz_hist = zz_hist[0]*1.3, zz_hist[1]
pp(zz_hist[0])

print('other:')
pp(other_hist[0])

print('comparing data to mc hist')
pp(other_hist[0] + zz_hist[0] + higgs_hist[0])
pp(data_hist[0])

## Plot the Invariant Mass Histogram

See how the Monte Carlo generated counts compare to data

In [ ]:
def plot_stacked_hist(zz_factor, higgs_factor):
    main_axes = plt.gca()
    main_axes.errorbar(x=bin_centers, y=data_hist[0], yerr=np.sqrt(data_hist[0]), fmt='ko', label='data')

    # https://matplotlib.org/3.3.0/api/_as_gen/matplotlib.pyplot.hist.html
    # https://matplotlib.org/3.3.0/gallery/statistics/histogram_multihist.html

    main_axes.hist(x=[other_x, zz_x, higgs_x, ], 
                   bins=n_bins, 
                   range=bin_range, 
                   weights=[other_weights, zz_weights*zz_factor, higgs_weights*higgs_factor], 
                   color=['mediumpurple', 'skyblue', 'red'], 
                   label=['other', 'zz', 'higgs'],
                   histtype='bar',
                   stacked=True
                  )

    plt.xlim(bin_range[0], bin_range[1])
    plt.ylim(0, 35)
    plt.xlabel('m_4l H->ZZ [GeV]')
    plt.ylabel('Events')
    plt.xticks(np.arange(bin_range[0], bin_range[1]+10, 10))
    plt.legend()

plot_stacked_hist(1.3, 1.0)

t2 = datetime.now()
print(t2)
print('total runtime: {} s'.format((t2-t1).seconds))

## Prepare the pyhf Workspace

Here we define the workspace which will eventually be used by pyhf to give the signal region normalization factor

In [ ]:
null = None
pyhf_workspace = \
{
    "channels": [
        {
            "name": "Signal_region",
            "samples": [
                {
                    "data": zz_hist[0].tolist(),
                    "modifiers": [
                        {
                            "data": zz_hist_err.tolist(),
                            "name": "staterror_Signal_region",
                            "type": "staterror"
                        }
                    ],
                    "name": "ZZ"
                },
                {
                    "data": higgs_hist[0].tolist(),
                    "modifiers": [
                        {
                            "data": higgs_hist_err.tolist(),
                            "name": "staterror_Signal_region",
                            "type": "staterror"
                        },
                        {
                            "data": null,
                            "name": "Signal_norm",
                            "type": "normfactor" # free parameter in fit
                        }
                    ],
                    "name": "Higgs"
                },
                {
                    "data": other_hist[0].tolist(),
                    "modifiers": [
                        {
                            "data": other_hist_err.tolist(),
                            "name": "staterror_Signal_region",
                            "type": "staterror"
                        }
                    ],
                    "name": "Other"
                }
            ]
        }
    ],
    "measurements": [
        {
            "config": {
                "parameters": [
                    {
                        "bounds": [
                            [
                                0,
                                5
                            ]
                        ],
                        "inits": [
                            1
                        ],
                        "name": "Signal_norm"
                    }
                ],
                "poi": "Signal_norm"
            },
            "name": "minimal_example"
        }
    ],
    "observations": [
        {
            "data": data_hist[0].tolist(),
            "name": "Signal_region"
        }
    ],
    "version": "1.0.0"
}

with open("higgs4l_pyhf_workspace.json", "w") as outfile:  
    json.dump(pyhf_workspace, outfile, indent=4) 

## Run pyhf Fit

After running the fit, extract the best fit normalization factor

In [ ]:
import fit
with open("higgs4l_pyhf_workspace.json") as f:
    ws = json.load(f)

bestfit, uncertainty, labels = fit.fit(ws)

for idx, label in enumerate(labels):
    if label == 'Signal_norm':
        bestfit_norm = bestfit[idx]
    else:
        continue

## Compare the Original and Normalized Signal Region Histograms

In [ ]:
plot_stacked_hist(zz_factor=1.3, higgs_factor=1.0)

In [ ]:
plot_stacked_hist(zz_factor=1.3, higgs_factor=bestfit_norm)

## What's Next?

- Integrate further with Cabinetry framework by using Awkward Arrays instead of Pandas
- Add Systematic Uncertainties (maybe dummy)
- Simultaneously fit zz normalization factor